In [64]:
import os
import pandas as pd

data_df = pd.DataFrame(columns=['text','Intent'])
filenames = list()
intervention_subgraphs = list()
for directory, subdirectories, files in os.walk('C:/Users/Tamir/Desktop/HSE_Project/excel_files/'):
    for file in files:
        filenames.append(os.path.join(directory,file))
for filename in filenames:
    if (filename[len(filename) - 22] == '~'):
        continue
    filename = filename.replace(" ", " ")
    post_df = pd.read_excel(filename)
    
        #fix to add columns if they are not in the xlsx file
    if 'attachments' not in post_df.columns:
        post_df['attachments'] = ''
    if 'reply_to_comment' not in post_df.columns:
        post_df['reply_to_comment'] = 0
    if 'reply_to_user' not in post_df.columns:
        post_df['reply_to_user'] = 0
        
    flex_df = post_df.drop(columns=['attachments','date','from_id','id','reply_to_comment','reply_to_user','Content','Intervention'])
    data_df = data_df.append(flex_df,ignore_index = True)

In [65]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
tokenizer = RegexpTokenizer(r'\w+')
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

def clean_text(text, remove_stopwords=True):
    """
    Removes stop words from the given text
    :param text: string with the text
    :param remove_stopwords: flag to enable removal of stop words
    :return: set of tokens without any stop words
    """
    text = text.lower()
    tokens = tokenizer.tokenize(text)
    tags = list()
    

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words('russian'))
        tokens = [w for w in tokens if w not in stops]
    for i in range(len(tokens)):
        tokens[i] = morph.parse(tokens[i])[0].normal_form
        t = morph.parse(tokens[i])[0]
        tokens[i] = tokens[i] + '_' + str(t.tag.POS)
    return tokens

delete_index = list()

for index,row in data_df.iterrows():
    if ( pd.isnull(data_df['text'].loc[index]) or pd.isnull(data_df['Intent'].loc[index]) 
              or (data_df['Intent'].loc[index] == ' -' ) ):
        delete_index.append(index)        
    else:
        data_df['text'].loc[index] = clean_text(data_df['text'].loc[index])

data_df = data_df.drop(index = delete_index)
data_df = data_df.reset_index(drop=True)

for index,row in data_df.iterrows():
    data_df['Intent'].loc[index] = data_df['Intent'].loc[index][0]
    if(data_df['Intent'].loc[index] == 'E'):
        print('change in ' + str(index))
        data_df['Intent'].loc[index] = 'Е'
        

            

change in 3158


In [97]:
#import json
#pymorphy_tags={
#        +'NOUN',  # имя существительное
#        +'ADJF',  # имя прилагательное (полное)
#        +'ADJS',  # имя прилагательное (краткое)
#        +'VERB',  # глагол (личная форма)
#        +'INFN',  # глагол (инфинитив)
#        +'PRTF',  # причастие (полное)
#        +'PRTS',  # причастие (краткое)
#        +'GRND',  # деепричастие
#        +'NUMR',  # числительное
#        +'ADVB',  # наречие
#        +'NPRO',  # местоимение-существительное
#        +'PRED',  # предикатив
#        +'PREP',  # предлог
#        +'CONJ',  # союз
#        +'PRCL',  # частица
#        +'INTJ',  # междометие
#}
#universal_tags={
#    +#ADJ: adjective
#    +#ADP: adposition
#    +#ADV: adverb
#    +#CCONJ: coordinating conjunction
#    +#DET: determiner
#    +#INTJ: interjection
#    +#NOUN: noun
#    +#NUM: numeral
#    +#PART: particle
#    +#PRON: pronoun
#    +#VERB: verb
#}

#Dict for tranformation pymorphy2 tags into universal POS tags

transform_dict = {'NOUN':'NOUN',
                  'ADJF':'ADJ',
                  'ADJS':'ADJ',
                  'PREP':'ADP',
                  'ADVB':'ADV',
                  'VERB':'VERB',
                  'INFN':'VERB',
                  'CONJ':'CCONJ',
                  'NPRO':'PRON',
                  'INTJ':'INTJ',
                  'PRCL':'PART',
                  'NUMR':'NUM',
                  'PRTF':'ADJ',
                  'PRTS':'ADJ',
                  'GRND':'ADJ',
                  'PRED':'ADJ'}

for index,row in data_df.iterrows():
    delete_list = list()
    #print('new comment')
    comment = data_df['text'].loc[index]
    for i in range(len(comment) - 1):
        #print('word: ' + comment[i])
        tag = comment[i].split('_')[len(comment[i].split('_'))-1]

        #print('old tag: ' + tag)
        if (tag == 'None'):
            delete_list.append(comment[i])
        else:
            tag = transform_dict.get(tag)
            #print('new tag: ' + tag)
            comment[i] = comment[i].split('_')[0] + '_' + tag
    for delete_value in delete_list:
        comment.remove(delete_value)



TypeError: must be str, not NoneType

In [69]:
def create_average_vec(doc, vec_dim, ft_model):
    average = np.zeros((vec_dim,), dtype='float32')
    num_words = 0.
    for word in doc:
        if word in ft_model.wv.vocab:
            average = np.add(average, ft_model[word])
            num_words += 1.
    if num_words != 0.:
        average = np.divide(average, num_words)
    return average

word_counter = 0
model_counter = 0
for index,row in data_df.iterrows():
    comment = data_df['text'].loc[index]
    for word in comment:
        word_counter += 1
        if (word in model.vocab):
            model_counter += 1
print('All the words in data after preprocessing: ' + str(word_counter))
print('Words in model vacabulary: ' + str(model_counter))

All the words in data after preprocessing: 47792
Words in model vacabulary: 36163


In [80]:
import numpy as np
vector_data = list()


for index,row in data_df.iterrows(): 
    comment = data_df['text'].loc[index]
    comment_vector = np.zeros((300,), dtype='float32')
    words = 0
    for word in comment:
        if (word in model.vocab):
            comment_vector = np.add(comment_vector,model[word])
            words += 1
    if (words != 0):
        comment_vector = np.divide(comment_vector,words)
    vector_data.append(comment_vector)

In [59]:
import gensim
import codecs
from gensim.models.wrappers import FastText
import gensim.downloader as api

#FastText.load_model('cc.ru.300')
model = api.load("word2vec-ruscorpora-300")

#w2v_file = codecs.open('cc.ru.300.bin.gz', encoding='utf-8')
#model = gensim.models.KeyedVectors.load_word2vec_format(w2v_file, binary=True)  # or binary=False if the model is not compressed

KeyboardInterrupt: 

In [95]:
import numpy as np
np.random.seed(123)  # for reproducibility
 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution1D, MaxPooling1D
from keras.utils import np_utils
from keras.datasets import mnist
from sklearn.model_selection import train_test_split

label_list = list()
for intent in data_df['Intent']:
    label_list.append(intent)

# 4. Load pre-shuffled MNIST data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(vector_data, label_list, test_size=0.2, random_state = 42, stratify = label_list)

 
# 5. Preprocess input data
#X_train = X_train.reshape(X_train.shape[0], 1, 28, 28)
#X_test = X_test.reshape(X_test.shape[0], 1, 28, 28)
#X_train = X_train.astype('float32')
#X_test = X_test.astype('float32')
#X_train /= 255
#X_test /= 255
 
# 6. Preprocess class labels
Y_train = np_utils.to_categorical(y_train, 25)
Y_test = np_utils.to_categorical(y_test, 25)
 
# 7. Define model architecture
model = Sequential()
 
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(25, activation='softmax'))
          

 
# 8. Compile model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
          
# 9. Fit model on training data
model.fit(X_train, Y_train, 
          batch_size=32, nb_epoch=10, verbose=1)
 
# 10. Evaluate model on test data
score = model.evaluate(X_test, Y_test, verbose=0)

ValueError: invalid literal for int() with base 10: 'Н'

In [96]:
#tags = list()
#for word in model.vocab:
    #print(word)

In [43]:
tokens = ['ага','держи','карман','шире']
for i in range(len(tokens)):
    print(tokens[i])
    tokens[i] = morph.parse(tokens[i])[0].normal_form
    t = morph.parse(tokens[i])[0]
    tokens[i] = tokens[i] + '_' + str(t.tag.POS)
    print(tokens[i])
print(tokens)


tokens = ['ага','держи','карман','шире']
for token in tokens:
    print(token)
    token = morph.parse(token)[0].normal_form
    t = morph.parse(token)[0]
    token = token+'_'+str(t.tag.POS)
    print(token)
print(tokens)

ага
ага_INTJ
держи
держать_INFN
карман
карман_NOUN
шире
шире_NOUN
['ага_INTJ', 'держать_INFN', 'карман_NOUN', 'шире_NOUN']
ага
ага_INTJ
держи
держать_INFN
карман
карман_NOUN
шире
шире_NOUN
['ага', 'держи', 'карман', 'шире']


In [1]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

word = 'дома'
word = morph.parse(word)[0].normal_form
t = morph.parse(word)[0]
word = word+'_'+str(t.tag.POS)

In [84]:
len(vector_data)

3661

In [91]:
data_df['Intent'].unique()

array(['Ц', 'Н', 'Ш', 'Ж', 'И', 'Ч', 'Т', 'Щ', 'А', 'Г', 'У', 'Х', 'Б',
       'Д', 'Л', 'В', 'Е', 'П', 'К', 'С', 'М', 'Ф', 'З', 'Р', 'О'],
      dtype=object)

In [37]:
!pip install pymorphy2

  Running setup.py bdist_wheel for docopt: started
  Running setup.py bdist_wheel for docopt: finished with status 'done'
  Stored in directory: C:\Users\Tamir\AppData\Local\pip\Cache\wheels\9b\04\dd\7daf4150b6d9b12949298737de9431a324d4b797ffd63f526e
Successfully built docopt


notebook 5.4.0 requires ipykernel, which is not installed.
jupyter 1.0.0 requires ipykernel, which is not installed.
jupyter-console 5.2.0 requires ipykernel, which is not installed.
ipywidgets 7.1.1 requires ipykernel>=4.5.1, which is not installed.
distributed 1.20.2 requires msgpack-python, which is not installed.
tensorflow 1.11.0 has requirement setuptools<=39.1.0, but you'll have setuptools 40.4.3 which is incompatible.
pyowm 2.8.0 has requirement requests<2.19,>=2.18.2, but you'll have requests 2.19.1 which is incompatible.
copydoc 1.0.9 has requirement beautifulsoup4==4.4.1, but you'll have beautifulsoup4 4.6.3 which is incompatible.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
